In [1]:
from xcube.core.store import new_data_store
from xcube_stac.utils import reproject_bbox

In [2]:
credentials = dict(
    key="",
    secret="",
)

In [3]:
store = new_data_store("stac-cdse-ardc", **credentials)

In [ ]:
team_store = new_data_store(
    "s3",
    root=os.environ["S3_USER_STORAGE_BUCKET"],
    storage_options=dict(
        anon=False,
        key=os.environ["S3_USER_STORAGE_KEY"],
        secret=os.environ["S3_USER_STORAGE_SECRET"],
    ),
)

In [4]:
store.list_data_ids()

['sentinel-2-l2a', 'sentinel-2-l1c', 'sentinel-3-syn-2-syn-ntc']

In [ ]:
bbox_wgs84 = []
crs_utm = "EPSG:"
bbox_utm = reproject_bbox(bbox_wgs84, "EPSG:4326", crs_utm)

In [ ]:
ds = store.open_data(
    "sentinel-2-l2a",
    time_range=("2025-06-01", "2025-12-31"),
    bbox=bbox_utm,
    spatial_res=10,
    crs=crs_utm,
    asset_names=["B02", "B03", "B04", "B08", "SCL"]
)
ds

In [ ]:
team_store.write_data(ds, "sen2l2a_stavanger_2022080712.zarr", replace=True)